In [7]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine # Needed for DB connection
from sqlalchemy import types
from configparser import ConfigParser

In [125]:
# Read the data
data = pd.read_csv('data_6.csv', sep=',', encoding='ISO-8859-1')
# Add the index of the matches
data['matchid'] = data.reset_index().index

# Clean the data
data = data[(data['a_score'] < 10) & (data['h_score'] < 10)]
data = data[(data['a_fouls'] < 30) & (data['h_fouls'] < 30)] 
data = data[(data['a_shot_acc'] <= 100) & (data['h_shot_acc'] <= 100)]
data = data[(data['a_pass_acc'] <= 100) & (data['h_pass_acc'] <= 100)]
data = data[(data['a_possession'] <= 100) & (data['h_possession'] <= 100)]

print(data.shape)

(506, 26)


In [126]:
# Create the team table
h_team = data[['h_team']]
h_team = h_team.rename(columns={'h_team':'team'})
a_team = data[['a_team']]
a_team = a_team.rename(columns={'a_team':'team'})

# Merges the teams in each team column
team_merged = pd.merge(h_team, a_team, on='team', how='outer')

# Remove all the duplicates
team_merged = team_merged.drop_duplicates(subset=['team'], keep='last')

# Add index to the team
team_merged['teamid'] = team_merged.reset_index().index

# Order the columns
team_merged = team_merged[['teamid','team']]

team_merged.head()

,teamid,team
0,0,ILikeUrTeamG
1,1,SculoAndry
2,2,Milan Ciuska
3,3,Gheorghe XI
643,4,FCT x BVB


In [127]:
team_merged.shape

(494, 2)

In [140]:
# Select the needed columns
match = data[['matchid','h_team', 'a_team', 'time', 'cancelled', 'filename','h_penalty_score','a_penalty_score']]

# Detect if a match had penalties or not
match['penalty'] = np.where(
    (match['h_penalty_score'] > 0) | (match['a_penalty_score'] > 0), 
    True, False)

# Merge ID of the home team
match = pd.merge(match, team_merged, left_on='h_team', right_on='team', how='inner')
match = match.drop(['team','h_team', 'h_penalty_score', 'a_penalty_score'], axis = 1)
match = match.rename(columns={'teamid':'homeid','filename':'file_name'})

# Merge ID of the away team
match = pd.merge(match, team_merged, left_on='a_team', right_on='team', how='inner')
match = match.drop(['team','a_team'], axis = 1)
match = match.rename(columns={'teamid':'awayid'})

# Re-order columns
match = match[['matchid','homeid','awayid','time','cancelled','file_name', 'penalty']]

match.head()

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_1568\3584960272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match['penalty'] = np.where(


,matchid,homeid,awayid,time,cancelled,file_name,penalty
0,1,0,4,83:08,True,10-11-2020_16-35-36-0ag1a4hv.png,False
1,2,1,4,90:00,False,10-11-2020_16-35-36-0wd3xedc.png,False
2,4,2,4,8:40,True,10-11-2020_16-35-37-pkztc23y.png,False
3,6,3,4,3:23,True,10-11-2020_16-35-39-gd3v30gx.png,False
4,8,5,4,90:00,False,10-11-2020_16-35-40-12q20rtg.png,False


In [129]:
penalty = data[['matchid', 'h_penalty_score','a_penalty_score']]
penalty = pd.merge(penalty, match, on='matchid', how='inner')
penalty = penalty[(penalty['penalty']) == True]
penalty = penalty.drop(['homeid','awayid','time','cancelled','file_name','penalty'], axis=1)

penalty.head()

,matchid,h_penalty_score,a_penalty_score
57,86,3.0,1.0
66,98,3.0,2.0
123,162,4.0,5.0
187,227,4.0,3.0
219,264,2.0,3.0


In [130]:
# Get the stats of either the home team or the away team
def get_stats(data, team_type='h'):
    # Get the required columns
    team_stats = data[['matchid', 
    '{}_team'.format(team_type), 
    '{}_score'.format(team_type),
    '{}_shots'.format(team_type), 
    '{}_shots_on_target'.format(team_type), 
    '{}_possession'.format(team_type), 
    '{}_tackles'.format(team_type), 
    '{}_fouls'.format(team_type),
    '{}_corners'.format(team_type),
    '{}_shot_acc'.format(team_type),
    '{}_pass_acc'.format(team_type)]]

    # Merge ID of the home team
    team_stats = pd.merge(team_stats, team_merged, left_on='{}_team'.format(team_type), right_on='team', how='inner')
    team_stats = team_stats.drop(['team','{}_team'.format(team_type)], axis = 1)
    
    # Rename the columns
    team_stats = team_stats.rename(columns={
        'matchid':'matchid',
        'id':'teamid',
        '{}_score'.format(team_type):'score',
        '{}_shots'.format(team_type):'shots',
        '{}_shots_on_target'.format(team_type):'shots_on_target',
        '{}_possession'.format(team_type):'possession',
        '{}_tackles'.format(team_type):'tackles',
        '{}_fouls'.format(team_type):'fouls',
        '{}_corners'.format(team_type):'corners',
        '{}_shot_acc'.format(team_type):'shot_accuracy',
        '{}_pass_acc'.format(team_type):'pass_accuracy'})

    # Re-order columns
    team_stats = team_stats[['matchid',
    'teamid',
    'score',
    'shots',
    'shots_on_target',
    'possession',
    'tackles',
    'fouls',
    'corners',
    'shot_accuracy',
    'pass_accuracy']]

    return team_stats

In [131]:
# Get the statistics for the home team
team_stats_home = get_stats(data, team_type='h')
# Get the statistics for the away team
team_stats_away = get_stats(data, team_type='a')

In [132]:
print(team_stats_away.shape)
team_stats_away.head()

(506, 11)


,matchid,teamid,score,shots,shots_on_target,possession,tackles,fouls,corners,shot_accuracy,pass_accuracy
0,1,4,-1,9,8,50,10,0,0,88,90
1,2,4,5,10,7,50,9,2,0,70,91
2,4,4,-1,3,3,64,0,0,0,100,92
3,6,4,-1,1,1,69,0,0,0,100,100
4,8,4,3,7,6,47,17,2,0,85,86


In [141]:
# Checks for outliers in the possessions columns of the dataset
#
# possessions = match['matchid']
# possessions = pd.merge(possessions, team_stats_away, on='matchid', how='outer')
# possessions = possessions.rename(columns={'possession':'a_possession'})
# possessions = pd.merge(possessions, team_stats_home, on='matchid', how='outer')
# possessions = possessions.rename(columns={'possession':'h_possession'})

# possessions = possessions[['matchid','h_possession','a_possession']]
# possessions['sum'] = possessions['h_possession'] + possessions['a_possession']
# possessions.loc[possessions['sum'] > 100].head()
# possessions.loc[possessions['sum'] < 100].head()


# possessions.head()

In [ ]:
# Write configuration data to file
#
# from configparser import ConfigParser
# config = ConfigParser()

# config.read('config.ini')
# config.add_section('main')
# config.set('main', 'driver', 'fill')
# config.set('main', 'username', 'fill')
# config.set('main', 'dbname', 'fill')
# config.set('main', 'password', 'fill')
# config.set('main', 'server', 'fill')
# config.set('main', 'port', 'fill')

# with open('config.ini', 'w') as f:
#     config.write(f)

In [8]:

def establish_connection():
    config = ConfigParser()
    config.read('config.ini')

    # Database settings
    driver = config.get('main', 'driver')
    username = config.get('main', 'username')
    dbname = config.get('main', 'dbname')
    password = config.get('main', 'password')
    server = config.get('main', 'server')
    port = config.get('main', 'port')

    # Creating the connection pool for SQL
    return create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')

def save_data(df, table_name, engine, schema_name = 'project'):
    # Saves the team table
    return df.to_sql(table_name, engine, schema=schema_name, index=False, if_exists='append', method='multi')

In [ ]:
engine = establish_connection()

# Save teams
save_data(team_merged, 'team', engine)
# Save matches
save_data(match, 'match', engine)
# Save home team matches
save_data(team_stats_home, 'team_stats', engine)
# Save away team matches
save_data(team_stats_away, 'team_stats', engine)
# Saves penalty table
save_data(penalty, 'penalty', engine)